<a id=index></a>
# Index
- <a href=#s3>S3</a>
- <a href=#cloudfront>CloudFront</a>

<a id=s3></a>
## S3
<a href=#index>Go to Index</a>

### Generic info
- ACL are used to control access to specific resources. Bucket policy applies to the whole bucket.
- When a file is uploaded to S3, the browser gets a 200 response for a successful upload. 
- All objects are private by default
- Min sized object on S3 is 0 bytes (eg: Delete marker)
- 3 types of storage - Standard, IA (Infrequently accessed), Reduced Redundancy Storage
- Can use client side or server side encryption. Server side encryption consists of 3 options -
	- S3 (SSE-S3)
	- KMS (SSE-KMS)
	- Customer provided keys (SSE-C)

### Versioning
- For a public file with versioning enabled, it will be made private in the new version. 
- Versioning cannot be removed (can be suspended)
- Actually when you delete a file, it gets another version called 'delete marker'. If you delete the delete marker, the file gets restored.

- The files only get removed when you got to versioning and remove the versions themselves
- There is also an option to enable MFA for deleting files with versioning enabled.

### Cross Region Replication (CRR)
- When CRR is newly introduced on a bucket, the existing objects are not automatically copied. New and updated objects are copied to the replication bucket. AWS CLI (s3 cp --recursive) can be used to copy all files from the primary to the replication bucket
- Versioning must be enabled (source and target) for CRR
- Src and target should be in different regions (Cross **Region** replication)
- At this time, you cannot do replication to multiple buckets (daisy chaining is not supported at present)
- Deleting files (delete marker) or deletion of individual versions are not replicated to the secondary buckets (this is an AWS security feature

### Lifecycle rule
- Main idea is to reduce cost
- S3 can automatically move contents of a bucket (or folder or things with a spefied tags ) to S3-IA and later to S3 Glacier based on what we set in the Management>Lifecycle tab added as Lifecycle rule. - Can also be configured to permanently delete stuff at some point
- To move to S3-IA requires a minimum of 30 days after creation and moving to Glacier requires 30 days after residing in IA or 60 days from creation
- Can be done along with versioning.

### S3 Security & Encryption
- By default all S3 objects are private

<a id=cloudfront></a>
## CloudFront
<a href=#index>Go to Index</a>

### Generic
- CDN - Content Delivery Network. Something that enables faster content delivery of content if the geographical location of the requestor and requested content might otherwise cause delay in service.
- Edge location (EL) is different from AWS region or AZ. Currently there are over 50 ELs. 
- Edge location takes content from origin. At this time origin can be 
	- S3 bucket
	- EC2 instance
	- Elastic Load Balancer
	- Route53
	- Can also be an external source of the files
- Distribution is the name given to the CDN which consists of a collection of Edge locations
- When content is requested first time, it is fetched from origin and then cached in the EL. This is cached for a specified TTL (Time To Live)
- There are two types of distribution
	- Web Distribution - Typically used for websites
	- RTMP - used for media streaming
- Edge locations are **not read-only**. They can also be used for put operations and that object will then be written back to the origin.
- You can clear the cached objects but you will be charged for it. The normal behavior is that the cached object will expire on its own after the specified TTL.
- Specifics of setting up the CloudFront Distribution
    - CloudFront uses an identity to access the S3 bucket (Origin Access Identity parameter in the CloudFront dist setup)
    - In the setup, remember to 'Grant Read permissions on bucket' to 'yes'
    - Allow HTTP and HTTPS(recommended for regular pages)
    - TTL is an important design consideration (set based on how frequently the objects change)
    - Pick the default SSL certificate so as to support HTTPS unless you can publish a certificate
    - There is a feature called Restrictions - allows Geo-Restrictions using a Whitelist and a Blacklist. Blacklist allows global access and restricts to few entities. Whitelist only serves to the entities in the whitelist.
    - "Invalidation" is the process of removing an object that is cached on CloudFront. Remember that this is to delete objects before TTL. *This costs money.*